In [49]:
import pandas as pd
import numpy as np
import datetime
import holidays
pd.options.display.float_format = '{:.2f}'.format

In [50]:
df = pd.read_csv("data/bikes.csv", index_col=0)

<p> Eliminamos la columna index al tratarse de un índice </p>

In [51]:
df.drop(['instant'], axis= 1, inplace= True)

<p> Cambiamos la columna dteday a datetime </p>

In [52]:
df["dteday"] = pd.to_datetime(df["dteday"], format='%d-%m-%Y')

In [53]:
df.head(15)

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,2018-01-01,spring,0,1,0,6,0,2,14.11,18.18,80.58,10.75,331,654,985
1,2018-01-02,spring,0,1,0,0,0,2,14.90,17.69,69.61,16.65,131,670,801
2,2018-01-03,spring,0,1,0,1,1,1,8.05,9.47,43.73,16.64,120,1229,1349
3,2018-01-04,spring,0,1,0,2,1,1,8.20,10.61,59.04,10.74,108,1454,1562
4,2018-01-05,spring,0,1,0,3,1,1,9.31,11.46,43.70,12.52,82,1518,1600
5,2018-01-06,spring,0,1,0,4,1,1,8.38,11.66,51.83,6.00,88,1518,1606
6,2018-01-07,spring,0,1,0,5,1,2,8.06,10.44,49.87,11.30,148,1362,1510
7,2018-01-08,spring,0,1,0,6,0,2,6.76,8.11,53.58,17.88,68,891,959
8,2018-01-09,spring,0,1,0,0,0,1,5.67,5.81,43.42,24.25,54,768,822
9,2018-01-10,spring,0,1,0,1,1,1,6.18,7.54,48.29,14.96,41,1280,1321


Recategorizamos la columna "yr"

In [54]:
df['yr'] = df['dteday'].apply(lambda x: x.year)

Recategorizamos la columna "mnth"

In [55]:
df['mnth'] = df['dteday'].apply(lambda x: x.strftime("%b"))

Recategorizamos la columna "weekday"

In [56]:
df['weekday'] = df['dteday'].apply(lambda x: x.strftime("%a"))

Recategorizamos la columna "season"

In [57]:
def estacion(fecha):
    
    #Establecemos los periodos de cada estación:
    primavera_inicio = pd.Timestamp(fecha.year, 3, 20)
    primavera_fin = pd.Timestamp(fecha.year, 6, 20)
    verano_inicio = pd.Timestamp(fecha.year, 6, 21)
    verano_fin = pd.Timestamp(fecha.year, 9, 22)
    otono_inicio = pd.Timestamp(fecha.year, 9, 23)
    otono_fin = pd.Timestamp(fecha.year, 12, 20)
    
    #hacemos la conversión de fecha a estación
    if primavera_inicio <= fecha <= primavera_fin:
        return "Primavera"
    elif verano_inicio <= fecha <= verano_fin:
        return "Verano"
    elif otono_inicio <= fecha <= otono_fin:
        return "Otoño"
    else:
        return "Invierno"

# Hacemos el apply:
df["season"] = df["dteday"].apply(estacion)

In [58]:
df.sample(10)

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
534,2019-06-19,Primavera,2019,Jun,0,Wed,1,1,28.22,32.73,69.00,9.92,968,5857,6825
454,2019-03-31,Primavera,2019,Mar,0,Sun,0,2,17.39,21.09,73.83,16.79,2301,3934,6235
589,2019-08-13,Verano,2019,Aug,0,Tue,1,1,29.55,33.11,54.54,9.13,1207,5676,6883
253,2018-09-11,Verano,2018,Sep,0,Tue,0,1,26.79,30.46,71.38,7.71,1633,3413,5046
72,2018-03-14,Invierno,2018,Mar,0,Wed,1,1,13.33,16.60,49.70,9.17,359,1687,2046
429,2019-03-06,Invierno,2019,Mar,0,Wed,1,1,10.59,12.75,45.67,13.46,221,3735,3956
386,2019-01-22,Invierno,2019,Jan,0,Tue,0,2,6.66,8.11,79.62,13.38,196,1781,1977
480,2019-04-26,Primavera,2019,Apr,0,Fri,1,2,20.43,24.18,75.67,11.83,475,4551,5026
658,2019-10-21,Otoño,2019,Oct,0,Mon,0,1,19.03,22.82,51.00,11.13,2132,4692,6824
103,2018-04-14,Primavera,2018,Apr,0,Sat,1,1,19.17,23.14,54.04,7.42,529,2738,3267


<p> Guardar los cambios en este archivo al final de la sesión </p>

In [59]:
df.to_pickle("data/bikes_limpieza.pkl")